# **<font color='crimson'>Алгоритм получения желаемых метрик A/B-теста для двусторонней гипотезы, проверяющей монету на "честность"</font>**

---

**Выполнил**: Юмаев Егор

---

## <font color='green'>**1 Моделирование поведения "честной" монеты с оценкой False Positive Rate**</font>

---

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

### <font color='green'>**1.1 Оценка False Positive Rate (1 - Specificity)**</font>

---

Поведение "честной" монеты можно описать с помощью биномиального распределения:

In [2]:
np.random.binomial(1, 0.5, size=10)

array([1, 1, 1, 0, 0, 1, 1, 0, 0, 0])

В качестве базового сценария рассмотрим подход, когда одному эксперименту будет соответствовать 10 бросков монеты.

Если убрать параметр size, переместив количество повторений на место единицы (1), то функция np.random.binomial будет возвращать количество положительных исходов из серии запусков (количество "орлов" в случае "честной" монеты):

In [3]:
np.random.binomial(10, 0.5)

5

Сформулируем **гипотезы** для исследуемой монеты:

* **Н0**: вероятность выпадения "орла" равна 50% (монета "честная")

* **Н1**: вероятность выпадения "орла" **не** равна 50% (монета **не** честная)

"Честная" монета или "не честная" определяется тем, насколько, например, при 10 бросках, количество положительных исходов отличается от 5 (50%).

Сделаем предположение, что если по итогам 10 бросков количество положительных исходов равно 5 (отклонение 0), 4 или 6 (отклонение 1), 3 или 7 (отклонение 2), 2 или 8 (отклонение 3), - все такие запуски будут свидетельствовать в пользу **Н0**.

Если по итогам 10 бросков количество положительных исходов равно 1 или 9 (отклонение 4), 0 или 10 (отклонение 5), - такие запуски будут свидетельствовать в пользу **Н1**.


Смоделируем 100_000 экспериментов, во время каждого эксперимента "честная" монета будет "подбрасываться" 10 раз с подсчетом количества положительных исходов. Затем оценим отклонения полученных исходов от 5. Вычислим процент ситуаций, когда отклонение от 5 больше или равно 4.

In [4]:
# моделируем 100_000 экспериментов
n = 100_000
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(10, 0.5))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [5]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

Каждая строчка датафрейма аккумулирует сведения о количестве выпавших "орлов" из 10 подбрасываний монеты.

In [6]:
# выведем последние 5 строк датафрейма
df.tail()

,0
99995,8
99996,7
99997,7
99998,3
99999,4


In [7]:
# вычисляем число отклонений для каждого эксперимента
df['deviation'] = abs(5 - df[0])
df.tail()

,0,deviation
99995,8,3
99996,7,2
99997,7,2
99998,3,2
99999,4,1


In [8]:
# вычислим процент ситуаций, когда отклонение от 5
# больше или равно 4
(df['deviation'] >= 4).mean()

0.02179

В 2.2% случаев фиксируются значительные отклонения от 5.

С самого начала эксперимента мы знаем, что монета является "честной", - это отражено в параметрах заданного биномиального распределения. Однако в 2.2% случаев "честная" в реальности монета может быть ошибочно признана "нечестной". Это и будет оценкой **False Positive Rate**.

### <font color='green'>**1.2 Оценка True Positive Rate (Sensitivity)**</font>

---

Чтобы оценить True Positive Rate, необходимо, чтобы монета была "немного нечестной". "Нечестность" можно определить в фунции np.random.binomial, задав вероятность выпадения положительного исхода, например, на уровне 0.6.

In [9]:
# зададим размер MDE, при добавлении которого к вероятности 0.5
# монета перестанет быть честной
# при заданном теоретическом распределении
mde = 0.1

In [10]:
# моделируем 100_000 экспериментов
n = 100_000
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(10, 0.5 + mde))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [11]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

In [12]:
# считаем отклонения
df['deviation'] = abs(5 - df[0])

В предыдущем пункте, когда вычислили долю экстремальных отклонений от значения 5, мы получили оценку **False Positive Rate**, т.к., моделируя эксперимент, изначально знали, что монета честная. Для этого вероятнотсь выпадения успешного исхода в функции np.random.binomial() была установлена на уровне 0.5.

Теперь, на этапе подготовки эксперимента, мы указали эту вероятность равной 0.6. С самого начала знаем, что монета нечестная. Поэтому вычисление доли экстремальных отклонений от значения 5 дает оценку **True Positive Rate**.

In [13]:
# вычислим процент ситуаций, когда отклонение от 5
# больше или равно 4
(df['deviation'] >= 4).mean()

0.04894

Только лишь в 5% случаев мы смогли нечестную в реальности монету назвать нечестной. **Вывод**: при столь небольшой выборке, которую задали в функции np.random.binomial(), выявить маленький эффект (MDE) правильно не получится. Минимально необходимая мощность в соответствии со сложившейся практикой должна быть 80% (на 75% больше полученного результата).

Повысим значение MDE и повторно рассчитаем мощность.

In [14]:
# моделируем 100_000 экспериментов
n = 100_000
mde = 0.2
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(10, 0.5 + mde))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [15]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

In [16]:
# считаем отклонения
df['deviation'] = abs(5 - df[0])

In [17]:
# вычислим процент ситуаций, когда отклонение от 5
# больше или равно 4
(df['deviation'] >= 4).mean()

0.15009

Увеличение MDE без увеличения размера выборки в функции np.random.binomial() позволило повысить мощность сразу в три раза.

Увеличить **True Positive Rate** можно, понизив **False Positive Rate**. Для этого необходимо понизить порог, после которого разница 5 и среднего значения каждого отдельного эксперимента признается экстремальной. Понизим порог с 4 до 3.

In [18]:
# вычислим процент ситуаций, когда отклонение от 5
# больше или равно 3 при MDE = 0.2
(df['deviation'] >= 3).mean()

0.38332

С пониженным порогом мощность выросла более чем в два раза, но все еще не дотягивает до общепринятого рубежа в 80%.

Однако, мы не можем только для **True Positive Rate** изменить порог. Порог для **False Positive Rate** тоже изменится.

Посчитаем, как изменится **False Positive Rate** с новым порогом.

In [19]:
# для False Positive Rate
# моделируем 100_000 экспериментов
n = 100_000
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(10, 0.5))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [20]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

In [21]:
# вычисляем число отклонений для каждого эксперимента
df['deviation'] = abs(5 - df[0])

In [22]:
# вычислим процент ситуаций, когда отклонение от 5
# больше или равно 3
(df['deviation'] >= 3).mean()

0.11145

Количество случаев, когда честная монета будет признана нечестной (**False Positive Rate**), выросло с 2.2% до 11.1%. Таким образом, мы можем пожертвовать точностью статистического теста для получения желаемого размера мощности.

Снова рассчитаем **True Positive Rate**. На этот раз MDE установим еще выше: 0.3.

In [23]:
# моделируем 100_000 экспериментов
n = 100_000
mde = 0.3
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(10, 0.5 + mde))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [24]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

In [25]:
# считаем отклонения
df['deviation'] = abs(5 - df[0])

In [26]:
# вычислим процент ситуаций, когда отклонение от 5
# больше или равно 3
(df['deviation'] >= 3).mean()

0.67735

Одновременное увеличение MDE и снижение порога позволило увеличить мощность теста до 67%. В 67% случаев нечестная монета называется нечестной. Однако размер MDE оставляет желать лучшего, но это лучший результат, какой мы можем получить при выборке size = 10 в функции np.random.binomial().

Для улучшения результатов эксперимента необходимо увеличить размер выборки.

Разработаем алгоритм, которая позволит получить стандартные alpha, beta и  power при MDE = 0.1 (10%).

## <font color='green'>**2 Алгоритм получения желаемых метрик A/B-теста, проверяющего монету на "честность"**</font>

---

Зафиксируем желаемые значения метрик A/B-теста (процедуры проверки монеты на "честность"):

* **MDE**: 10%

* **Power (1 - beta)**: 80%

* **Significance (alpha)**: 5%

Требуется подобрать:

* **размер выборки**

* **отклонение от нормы**.

Начнем с подбора отклонения от нормы. Для этого зафиксируем первоначальный размер выборки на уровне 100 (по итогам ранее проведенных экспериментов мы знаем, что размер выборки 10 точно недостаточен).

### <font color='green'>**2.1 Подбор FPR (Significance) (1-я итерация)**</font>

---

In [27]:
# размер выборки
sample_size = 100

In [28]:
# для False Positive Rate
# моделируем 100_000 экспериментов
n = 100_000
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(sample_size, 0.5))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [29]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

Т.к. размер выборки изменился, теперь величину 10 необходимо заменить на размер sample_size, умноженный на необходимый процент.

In [30]:
# вычисляем число отклонений для каждого эксперимента
df['deviation'] = abs(sample_size * 0.5 - df[0])

In [31]:
# вычислим процент ситуаций, когда отклонение от 5
# больше или равно вручную подобранному порогу
(df['deviation'] >= sample_size * 0.11).mean()

0.03379

При размере выборки, равной 100 и пороге отклонения больше либо равном 11 **FPR (Significance)** равен **3.4%**.

### <font color='green'>**2.2 Подбор TPR  и MDE (1-я итерация)**</font>

---

Подбирать TPR и MDE будем одновременно, т.к. они связаны.

Порог отклонения зададим равным значению, подобранному в предыдущем пункте: sample_size * 0.11.

In [32]:
# моделируем 100_000 экспериментов
n = 100_000
mde = 0.1
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(sample_size, 0.5 + mde))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [33]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

In [34]:
# считаем отклонения
df['deviation'] = abs(sample_size * 0.5 - df[0])

In [35]:
# вычислим процент ситуаций, когда отклонение от 5
# больше или равно вручную подобранному порогу
(df['deviation'] >= sample_size * 0.11).mean()

0.46202

Получена мощность, равная 46%. Этого недостаточно, т.к. запланировано значение мощности в 80%.

Очевидно, необходимо увеличить размер выборки. Релаизуем вторую итерацию подбора желаемых значений метрик A/B-теста.

### <font color='green'>**2.3 Подбор FPR (Significance) (2-я итерация)**</font>

---

In [36]:
# размер выборки увеличиваем со 100 до 200
sample_size = 200

In [37]:
# для False Positive Rate
# моделируем 100_000 экспериментов
n = 100_000
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(sample_size, 0.5))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [38]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

Т.к. размер выборки изменился, теперь величину 10 необходимо заменить на размер sample_size, умноженный на необходимый процент.

In [39]:
# вычисляем число отклонений для каждого эксперимента
df['deviation'] = abs(sample_size * 0.5 - df[0])

In [40]:
# вычислим отклонение от нормы
threshold = sample_size * 0.07
threshold

14.000000000000002

In [41]:
# вычислим процент ситуаций, когда отклонение от sample_size * 0.5
# больше или равно threshold
(df['deviation'] >= threshold).mean()

0.04011

При размере выборки, равной 200 и пороге отклонения больше либо равном 14 **FPR (Significance)** равен **4%**.

### <font color='green'>**2.4 Подбор TPR  и MDE (2-я итерация)**</font>

---

Порог отклонения зададим равным значению, подобранному в предыдущем пункте: threshold.

In [42]:
# моделируем 100_000 экспериментов
n = 100_000
mde = 0.1
result = []

for i in tqdm(range(n)):
    np.random.seed(i) # воспроизводимость результатов
    result.append(np.random.binomial(sample_size, 0.5 + mde))

  0%|          | 0/100000 [00:00<?, ?it/s]

In [43]:
# сохраняем результаты в датафрейм
df = pd.DataFrame(result)

In [44]:
# считаем отклонения
df['deviation'] = abs(sample_size * 0.5 - df[0])

In [45]:
# вычислим процент ситуаций, когда отклонение от sample_size * 0.5
# больше или равно threshold
(df['deviation'] >= threshold).mean()

0.78681

Получена мощность, равная 78%. Этого недостаточно, т.к. запланировано значение мощности в 80%.

Однако, на 2-й итерации получено значение мощности, достаточно близкое к желаемому. Добиться мощности в 80% можно, регулируя объем выборки, но менять ее теперь необходимо незначительно, т.к. мы вплотную подошли к желаемой мощности, а значения Significance и MDE соответствуют запланированным.

# **<font color='royalblue'>Выводы</font>**

---

(1) С целью подбора желаемых метрик A/B-теста для **двусторонней** гипотезы были зафиксированы следующие значения метрик:

* **MDE**: 10%

* **Power (1 - beta)**: 80%

* **Significance (alpha)**: 5%

(2) В результате проведения ряда итераций подобраны:

* **размер выборки**

* **отклонение от нормы**


(3) Алгоритм подбора параметров определен следующим образом:

* **Отклонение от нормы** (пороговое значение) на каждой итерации корректируется таким образом, чтобы значение **Significance (alpha)** было максимально приближено к 0.05 (5%).

* Оптимальный, наиболее эффективный **размер выборки** определялся таким образом, чтобы подобрать наименьший размер выборки, позволяющий получить желаемые размеры метрик A\B-теста, - для снижения стоимости и длительности проведения теста.


(4) В отчете представлена схема алгоритма получения желаемых метрик A/B-теста на этапе планирования эксперимента.